In [0]:
%sql
  DROP TABLE  IF EXISTS table_train;


In [0]:
%sql
create table table_train
using csv
OPTIONS(path '/FileStore/tables/landing/train.csv', header 'true')

In [0]:
%sql
select * from table_train limit 6

id Gender Age Driving_License Region_Code Previously_Insured Vehicle_Age Vehicle_Damage Annual_Premium Policy_Sales_Channel Vintage Response 1 Male 44 1 28.0 0 > 2 Years Yes 40454.0 26.0 217 1 2 Male 76 1 3.0 0 1-2 Year No 33536.0 26.0 183 0 3 Male 47 1 28.0 0 > 2 Years Yes 38294.0 26.0 27 1 4 Male 21 1 11.0 1 < 1 Year No 28619.0 152.0 203 0 5 Female 29 1 41.0 1 < 1 Year No 27496.0 152.0 39 0 6 Female 24 1 33.0 0 < 1 Year Yes 2630.0 160.0 176 0

###numero de linhas

In [0]:
%sql
select count(id) from table_train  



count(id) 381109

### Consultando  o ID para ver se temos eles duplos, comparando com o numero de linhas

In [0]:
%sql
SELECT DISTINCT  count(id)
FROM table_train;

count(id) 381109

In [0]:
%sql
SELECT
   id, 
   COUNT(*) AS anzahl
FROM table_train
GROUP BY id
HAVING anzahl > 1


id anzahl

###Consultando as colunas da tabela para ver se  tem valores null ou vazio

In [0]:
spark.sql("SELECT * FROM table_train where Age IS NULL OR GENDER IS NULL OR Driving_License IS NULL OR Region_Code IS NULL ").show()

+---+------+---+---------------+-----------+------------------+-----------+--------------+--------------+--------------------+-------+--------+
| id|Gender|Age|Driving_License|Region_Code|Previously_Insured|Vehicle_Age|Vehicle_Damage|Annual_Premium|Policy_Sales_Channel|Vintage|Response|
+---+------+---+---------------+-----------+------------------+-----------+--------------+--------------+--------------------+-------+--------+
+---+------+---+---------------+-----------+------------------+-----------+--------------+--------------+--------------------+-------+--------+



In [0]:
spark.sql("SELECT * FROM table_train where Previously_Insured IS NULL OR Vehicle_Age IS NULL OR Vehicle_Damage IS NULL OR Annual_Premium IS NULL ").show()

+---+------+---+---------------+-----------+------------------+-----------+--------------+--------------+--------------------+-------+--------+
| id|Gender|Age|Driving_License|Region_Code|Previously_Insured|Vehicle_Age|Vehicle_Damage|Annual_Premium|Policy_Sales_Channel|Vintage|Response|
+---+------+---+---------------+-----------+------------------+-----------+--------------+--------------+--------------------+-------+--------+
+---+------+---+---------------+-----------+------------------+-----------+--------------+--------------+--------------------+-------+--------+



In [0]:

spark.sql("SELECT * FROM table_train where Policy_Sales_Channel IS NULL OR Vintage IS NULL OR Vehicle_Damage IS NULL OR Response IS NULL").show()

+---+------+---+---------------+-----------+------------------+-----------+--------------+--------------+--------------------+-------+--------+
| id|Gender|Age|Driving_License|Region_Code|Previously_Insured|Vehicle_Age|Vehicle_Damage|Annual_Premium|Policy_Sales_Channel|Vintage|Response|
+---+------+---+---------------+-----------+------------------+-----------+--------------+--------------+--------------------+-------+--------+
+---+------+---+---------------+-----------+------------------+-----------+--------------+--------------+--------------------+-------+--------+



In [0]:
%sql
SHOW COLUMNS FROM table_train

col_name id Gender Age Driving_License Region_Code Previously_Insured Vehicle_Age Vehicle_Damage Annual_Premium Policy_Sales_Channel Vintage Response

In [0]:
df_train = spark\
           .read.format('csv')\
           .option('InferSchema','true')\
           .option('header','true')\
           .csv('/FileStore/tables/landing/train.csv')

In [0]:
df_train.printSchema()

root
 |-- id: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Driving_License: integer (nullable = true)
 |-- Region_Code: double (nullable = true)
 |-- Previously_Insured: integer (nullable = true)
 |-- Vehicle_Age: string (nullable = true)
 |-- Vehicle_Damage: string (nullable = true)
 |-- Annual_Premium: double (nullable = true)
 |-- Policy_Sales_Channel: double (nullable = true)
 |-- Vintage: integer (nullable = true)
 |-- Response: integer (nullable = true)



In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType, DoubleType, TimestampType
from pyspark.sql.functions import col , regexp_replace
 

### Transformando as Colunas  "Vehicle_Age", 'Vehicle_Damage' e "Gender" em Integer

In [0]:
df_train = df_train.withColumn("Vehicle_Age",col("Vehicle_Age").cast(IntegerType()))



In [0]:
df_train = df_train.withColumn('Vehicle_Damage',col('Vehicle_Damage').cast(IntegerType()))

In [0]:
df_train = df_train.withColumn("Gender",col("Gender").cast(IntegerType()))
 ### Preparando a base de dados para treinamento
#df['Gender'] = df['Gender'].map( {'Female': 0, 'Male': 1} ).astype(int)
#df['Vehicle_Age'] = df['Vehicle_Age'].map( {'< 1 Year': 0, '1-2 Year': 1, '> 2 Years': 2 } ).astype(int)
#df['Vehicle_Damage'] = df['Vehicle_Damage'].map( {'No': 0, 'Yes': 1} ).astype(int)
#df.head()

In [0]:
df_train.printSchema()

root
 |-- id: integer (nullable = true)
 |-- Gender: integer (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Driving_License: integer (nullable = true)
 |-- Region_Code: double (nullable = true)
 |-- Previously_Insured: integer (nullable = true)
 |-- Vehicle_Age: integer (nullable = true)
 |-- Vehicle_Damage: integer (nullable = true)
 |-- Annual_Premium: double (nullable = true)
 |-- Policy_Sales_Channel: double (nullable = true)
 |-- Vintage: integer (nullable = true)
 |-- Response: integer (nullable = true)

